In [1]:
# Author: Selma Gomez Orr <selmagomezorrds@gmail.com> January 30, 2016

##########################################################################
## Imports
##########################################################################

import requests, urllib2, gzip, urllib
import pandas as pd
import numpy as np
import json
import ujson
import dill


import numpy as np

##########################################################################
## Constants
##########################################################################

url = "http://thedataincubator.s3.amazonaws.com/coursedata/mldata/yelp_train_academic_dataset_review.json.gz"

url2 = "http://thedataincubator.s3.amazonaws.com/coursedata/mldata/yelp_train_academic_dataset_business.json.gz"

##########################################################################
## Program - Miniproject: nlp Bag of Words
##########################################################################


#Download reviews file
urllib.urlretrieve(url,'Reviews')

reviews_list = []
with gzip.open('Reviews', 'rb') as f:
    for line in f.readlines():
        review = ujson.loads(line)
        reviews_list.append(review)
        
#print reviews_list[0:2]

#Download all companies file fromw which only restaurants will be extracted.
urllib.urlretrieve(url2, "Restaurants")
        
restaurants_list = []
with gzip.open('Restaurants', 'rb') as f:
    for line in f:
        restaurant = ujson.loads(line)
        if 'Restaurants' in restaurant['categories']:
            restaurants_list.append(restaurant)


print restaurants_list[0:1]

[{u'city': u'De Forest', u'review_count': 26, u'name': u'Pine Cone Restaurant', u'neighborhoods': [], u'type': u'business', u'business_id': u'JwUE5GmEO-sH1FuwJgKBlQ', u'full_address': u'6162 US Highway 51\nDe Forest, WI 53532', u'hours': {}, u'state': u'WI', u'longitude': -89.335844, u'stars': 4.0, u'latitude': 43.238893, u'attributes': {u'Take-out': True, u'Price Range': 1, u'Outdoor Seating': False, u'Caters': False, u'Noise Level': u'average', u'Parking': {u'garage': False, u'street': False, u'validated': False, u'lot': True, u'valet': False}, u'Delivery': False, u'Attire': u'casual', u'Has TV': True, u'Good For': {u'dessert': False, u'latenight': False, u'lunch': True, u'dinner': False, u'brunch': False, u'breakfast': False}, u'Takes Reservations': False, u'Ambience': {u'romantic': False, u'intimate': False, u'touristy': False, u'hipster': False, u'divey': False, u'classy': False, u'trendy': False, u'upscale': False, u'casual': False}, u'Waiter Service': True, u'Accepts Credit Card

In [2]:
#Create a dataframe with the text and business_id from the reviews for join.
fields = ['text', 'business_id']

df_reviews = pd.DataFrame(reviews_list, columns = fields)

df_reviews.head()
#df_reviews.describe()



,text,business_id
0,dr. goldberg offers everything i look for in a...,vcNAWiLM4dR7D2nwwJ7nCA
1,"Unfortunately, the frustration of being Dr. Go...",vcNAWiLM4dR7D2nwwJ7nCA
2,Dr. Goldberg has been my doctor for years and ...,vcNAWiLM4dR7D2nwwJ7nCA
3,Been going to Dr. Goldberg for over 10 years. ...,vcNAWiLM4dR7D2nwwJ7nCA
4,Got a letter in the mail last week that said D...,vcNAWiLM4dR7D2nwwJ7nCA


In [3]:
#Create a dataframe with the category and business_id from the business info for join.

fields2 = ['business_id', 'categories']

df_restaurants = pd.DataFrame(restaurants_list, columns = fields2)

df_restaurants.head()
#df_restaurants.describe() 

,business_id,categories
0,JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants]
1,uGykseHzyS5xAMWoN6YUqA,"[American (Traditional), Restaurants]"
2,LRKJF43s9-3jG9Lgx4zODg,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R..."
3,RgDg-k9S5YD_BaxMckifkg,"[Chinese, Restaurants]"
4,_wZTYYL7cutanzAnJUTGMA,"[Bars, American (Traditional), Nightlife, Loun..."


In [4]:
#Merge the reviews dataframe with the restaurants dataframe to get only reviews of restaurants.
df_food = pd.merge(df_reviews, df_restaurants, on = 'business_id')

df_food.head()
#df_food.describe()



,text,business_id,categories
0,Good truck stop dining at the right price. We ...,JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants]
1,"If you like lot lizards, you'll love the Pine ...",JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants]
2,Enjoyable experience for the whole family. The...,JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants]
3,One of my favorite truck stop diners with soli...,JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants]
4,Only went here once about a year and a half ag...,JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants]


In [5]:
#Select out the text for those reviews determined to be from restauarnts.

text = np.array(df_food['text'])

print len(text)
print text [1:5]

574278
[u"If you like lot lizards, you'll love the Pine Cone!"
 u'Enjoyable experience for the whole family. The wait staff was courteous and friendly; the food was reasonably priced and a good value. A word of advice--LEAVE ROOM FOR DESSERT! the deserters are great but huge! Plan to bring some home'
 u"One of my favorite truck stop diners with solid food and friendly, quick service. My god, those desserts are huge! I can't imagine eating that giant cream puff. \nAll the food we had was delicious and I love how they leave a carafe of coffee on the table. Love this place. Would definitely be back if I was in the area!"
 u"Only went here once about a year and a half ago, but they had great pancakes! My only problem with it at the time was that they allowed smoking, so I left smelling like a cigarette. With the change in law, I'm sure the atmosphere has improved!"]


In [6]:
#Create the vectors for single words.
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

vec1 = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'))
vec1_mat = vec1.fit_transform(text)
vec1_mat.shape



(574278, 173613)

In [9]:
#Create the vector for the word-pairs.
vec2 = CountVectorizer(ngram_range= (2,2), stop_words=nltk.corpus.stopwords.words('english'))
vec2_mat = vec2.fit_transform(text)
vec2_mat.shape


(574278, 7811592)

In [10]:
#Store instance of the matrices using dill.

with open('vec1_file','w') as f:
    dill.dump(vec1_mat,f)
    
with open('vec2_file', 'w') as f:
    dill.dump(vec2_mat, f)
    
with open('vec1_ojb_file', 'w') as f:
    dill.dump(vec1, f)
    
with open('vec2_obj_file', 'w') as f:
    dill.dump(vec2, f)


In [12]:
import dill 

#Load the instance of the matrices using dill.
vec1_mat = dill.load(open('vec1_file', 'r'))
vec2_mat = dill.load(open('vec2_file', 'r'))
vec1 = dill.load(open('vec1_ojb_file', 'r'))
vec2 = dill.load(open('vec2_obj_file', 'r'))

#Check shape.
print vec1_mat.shape
print vec2_mat.shape


(574278, 173613)
(574278, 7811592)


In [13]:
from scipy.sparse import *

#Get total count of all words for each matrix.
total1 = csr_matrix.sum(vec1_mat)
total2 = csr_matrix.sum(vec2_mat)

print total1
print total2

37171596
36597378


In [14]:

#Get total count for each word or pair of words.
count1 = np.array(csr_matrix.sum(vec1_mat, 0)[0])[0]
count2 = np.array(csr_matrix.sum(vec2_mat, 0)[0])[0]

print len(count1)
print len(count2)
print count1[1:5]
print count2[1:5]


173613
7811592
[623   1   1   1]
[1 1 1 8]


In [15]:
#Get the vocabularies for each vector
vocab1 = vec1.get_feature_names()
vocab2 = vec2.get_feature_names()

print len(vocab1)
print len(vocab2)
print vocab1[1:5]
print vocab2[1:5]



173613
7811592
[u'000', u'0000', u'000000', u'0000000']
[u'00 000', u'00 00am', u'00 00p', u'00 00pm']


In [16]:
#Create a list of tuples of the word or word-pair and the count
data1 = zip(vocab1, count1)
data2 = zip(vocab2, count2)

print len(data1)
print len(data2)

print data1[1:5]
print data2[1:5]


173613
7811592
[(u'000', 623), (u'0000', 1), (u'000000', 1), (u'0000000', 1)]
[(u'00 000', 1), (u'00 00am', 1), (u'00 00p', 1), (u'00 00pm', 8)]


In [17]:
#Create a dictionary with probabilities.

prob1_dict = {}
for item in data1:
    prob1_dict[item[0]] = item[1]/float(total1)

prob2_dict ={}
for elem in data2:
    prob2_dict[elem[0]] = elem[1]/float(total2)
    



In [22]:
#Create a dictionary with the calculations.
alpha = 2.*10**-19
calc_list = []

for entry in prob2_dict:
    pair = entry.split()
    p1 = pair[0]
    p2 = pair[1]
    food_gram = p1+" "+p2
    calc = (prob2_dict[entry] + alpha)/((prob1_dict[p1] *prob1_dict[p2]) + (alpha*total1))
    calc_list.append((calc, [food_gram]))

print len(calc_list)

print calc_list[1:5]


                                        

7811592
[(346.73015346408044, [u'babystack cafe']), (0.088804332115599183, [u'soggy said']), (0.29196273479680451, [u'service deliciousness']), (4.646536946397064, [u'siracha enjoy'])]


In [26]:

final_list = sorted(calc_list)
top = final_list[-100:]


In [31]:
bigrams_list = []
for gram in top:
    bigrams_list.append(gram[1])

print bigrams_list

[[u'upward projects'], [u'homer simpson'], [u'ama ebi'], [u'salo salo'], [u'hob nobs'], [u'turo turo'], [u'tsk tsk'], [u'fritto misto'], [u'demi glace'], [u'bla bla'], [u'highs lows'], [u'hon machi'], [u'val vista'], [u'goi cuon'], [u'bai thong'], [u'2weqs rnoobhb1kshkyosq'], [u'fra diavolo'], [u'harry potter'], [u'artery clogging'], [u'bells whistles'], [u'sais quoi'], [u'cullen skink'], [u'jap chae'], [u'itsy bitsy'], [u'betty boop'], [u'rx boiler'], [u'chino bandido'], [u'womp womp'], [u'ghillie dhu'], [u'shiner bock'], [u'yada yada'], [u'valle luna'], [u'yadda yadda'], [u'woonam jung'], [u'mccormick schmick'], [u'rustler rooste'], [u'sous vide'], [u'malai kofta'], [u'porta alba'], [u'ritz carlton'], [u'wal mart'], [u'aguas frescas'], [u'dueling pianos'], [u'kao tod'], [u'mille feuille'], [u'perrier jouet'], [u'khao soi'], [u'bradley ogden'], [u'ak yelpcdn'], [u'barnes noble'], [u'patatas bravas'], [u'lomo saltado'], [u'haricot vert'], [u'luc lac'], [u'bandeja paisa'], [u'dol sot'],